In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# MODEL 1
🔽 Encoder

Conv2d + ReLU

MaxPool2d (downsampling)

Boyut: 128 → 64 → 32

🔼 Decoder

Upsample (nearest neighbor)

Conv2d + ReLU

Boyut: 32 → 64 → 128

🎯 Öğrenme

Supervised Learning

Loss: MSELoss

Optimizer: Adam

1️⃣ GEREKLİ KÜTÜPHANELER

In [2]:
 import os
import random
import shutil
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torchvision.transforms.functional as TF


2️⃣ SADECE 500 ADET CELEBA GÖRSEL SEÇ

In [3]:
# CelebA orijinal klasörü
image_dir = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"

# Tüm görseller
all_images = os.listdir(image_dir)

# Tekrarlanabilirlik
random.seed(42)

# Sadece 500 görsel seç
selected_images = random.sample(all_images, 500)

# Hedef klasör
target_dir = "/kaggle/working/celeba_500/images"
os.makedirs(target_dir, exist_ok=True)

# Seçilen görselleri kopyala
for img_name in selected_images:
    shutil.copy(
        os.path.join(image_dir, img_name),
        os.path.join(target_dir, img_name)
    )

print("✅ 500 görsel hazır")


✅ 500 görsel hazır


3️⃣ DATASET (LR → HR PAIR)

In [4]:
class SuperResolutionDataset(Dataset):
    def __init__(self, image_dir, hr_size=128, scale=4):
        self.image_dir = image_dir
        self.images = os.listdir(image_dir)

        # HR (gerçek yüksek çözünürlük)
        self.hr_transform = transforms.Compose([
            transforms.Resize((hr_size, hr_size)),
            transforms.ToTensor()
        ])

        # LR (bilerek boz → küçült → tekrar büyüt)
        self.lr_transform = transforms.Compose([
            transforms.Resize((hr_size // scale, hr_size // scale)),
            transforms.Resize((hr_size, hr_size)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.open(
            os.path.join(self.image_dir, self.images[idx])
        ).convert("RGB")

        hr = self.hr_transform(img)  # hedef
        lr = self.lr_transform(img)  # giriş

        return lr, hr


4️⃣ DATASET & DATALOADER

In [5]:
dataset = SuperResolutionDataset("/kaggle/working/celeba_500/images")

train_size = int(0.7 * len(dataset))
val_size   = int(0.15 * len(dataset))
test_size  = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)


5️⃣ AUTOENCODER SUPER-RESOLUTION MODEL

In [6]:
# ============================================================
# AUTOENCODER SR – POOLING TABANLI (İLK MODEL)
# Tek görsel test: LR / SR / HR çıktıları
# ============================================================

import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# =========================
# MODEL (İLK MODEL – POOLING)
# =========================
class AutoencoderSR(nn.Module):
    def __init__(self):
        super().__init__()

        # -------- Encoder --------
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),      # 128 → 64

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)       # 64 → 32
        )

        # -------- Decoder --------
        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor=2),   # 32 → 64
            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),   # 64 → 128
            nn.Conv2d(64, 3, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

# =========================
# MODEL YÜKLE
# =========================
model = AutoencoderSR().to(device)
model.load_state_dict(torch.load("/kaggle/working/autoencoder_sr.pth", map_location=device))
model.eval()

# =========================
# TEST EDİLECEK RESİM
# (İSTEDİĞİN RESMİ BURAYA KOY)
# =========================
img_path = "/kaggle/working/celeba_500/images/003540.jpg"
img = Image.open(img_path).convert("RGB")

# =========================
# HR ve LR OLUŞTUR
# =========================
hr = T.Resize((128, 128))(img)
hr = TF.to_tensor(hr)

lr = TF.resize(hr, [32, 32])      # Modelin gördüğü giriş
lr_input = lr.unsqueeze(0).to(device)

# =========================
# SUPER RESOLUTION
# =========================
with torch.no_grad():
    sr = model(lr_input)

# =========================
# GÖRSELLEŞTİRME
# =========================
plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.imshow(lr.permute(1,2,0))
plt.title("Low Resolution (32x32)")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(sr.squeeze(0).permute(1,2,0).cpu())
plt.title("Super Resolved Output")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(hr.permute(1,2,0))
plt.title("Ground Truth (128x128)")
plt.axis("off")

plt.tight_layout()
plt.show()


Device: cuda


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/autoencoder_sr.pth'

6️⃣ TRAINING SETUP

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoencoderSR().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


7️⃣ TRAIN + VALIDATION LOOP

In [ ]:
num_epochs = 30
train_losses, val_losses = [], []

for epoch in range(num_epochs):

    # ---- TRAIN ----
    model.train()
    train_loss = 0

    for lr, hr in train_loader:
        lr, hr = lr.to(device), hr.to(device)

        out = model(lr)
        loss = criterion(out, hr)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # ---- VALIDATION ----
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for lr, hr in val_loader:
            lr, hr = lr.to(device), hr.to(device)
            val_loss += criterion(model(lr), hr).item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train: {train_loss:.4f} | Val: {val_loss:.4f}")


8️⃣ LOSS GRAFİĞİ

In [ ]:
plt.plot(train_losses, label="Train")
plt.plot(val_losses, label="Validation")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Progress")
plt.show()


9️⃣ MODELİ KAYDET

In [ ]:
torch.save(model.state_dict(), "/kaggle/working/autoencoder_sr.pth")
print("✅ Model kaydedildi")


In [ ]:
# ============================================================
# AUTOENCODER SUPER RESOLUTION - POOLING BASED (FIRST MODEL)
# Encoder: MaxPooling
# Decoder: Upsample
# Arayüz YOK – Manuel resim test
# ============================================================

import torch
import torch.nn as nn
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
from PIL import Image

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# =========================
# MODEL TANIMI (POOLING)
# =========================
class AutoencoderSR_Pooling(nn.Module):
    def __init__(self):
        super().__init__()

        # -------- ENCODER --------
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),          # 32 → 16

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)           # 16 → 8
        )

        # -------- DECODER --------
        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor=2),  # 8 → 16
            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),  # 16 → 32
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),  # 32 → 64
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),  # 64 → 128
            nn.Conv2d(32, 3, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

# =========================
# MODEL YÜKLE
# =========================
model_path = "/kaggle/working/autoencoder_sr.pth"  # ← pooling modelin
model = AutoencoderSR_Pooling().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# =========================
# TEST EDİLECEK RESİM
# =========================
test_img_path = "/kaggle/working/celeba_500/images/003540.jpg"  # ← istediğin resim
img = Image.open(test_img_path).convert("RGB")

# =========================
# HR / LR OLUŞTUR
# =========================
hr = TF.to_tensor(T.Resize((128, 128))(img))
lr = TF.resize(hr, [32, 32])

lr_input = lr.unsqueeze(0).to(device)

# =========================
# SUPER RESOLUTION
# =========================
with torch.no_grad():
    sr = model(lr_input)

# =========================
# GÖRSELLEŞTİRME
# =========================
plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.imshow(lr.permute(1,2,0))
plt.title("Low Resolution (32x32)")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(sr.squeeze(0).permute(1,2,0).cpu())
plt.title("Super Resolved (Pooling Model)")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(hr.permute(1,2,0))
plt.title("Original / Ground Truth (128x128)")
plt.axis("off")

plt.tight_layout()
plt.show()



🔟 GRADIO ARAYÜZÜ

In [ ]:
!pip install gradio

In [ ]:
import torch
import torchvision.transforms.functional as TF
import gradio as gr

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =========================
# MODELİ YÜKLE
# =========================
model = AutoencoderSR().to(device)

model.load_state_dict(
    torch.load("/kaggle/working/autoencoder_sr.pth", map_location=device)
)

model.eval()

# =========================
# SUPER RESOLUTION FONKSİYONU
# =========================
def super_resolve(image):
    # PIL → Tensor (HR)
    hr = TF.to_tensor(image)

    # Bilerek düşür (LR simülasyonu)
    lr = TF.resize(hr, [32, 32])

    # Modele uygun hale getir
    lr_up = TF.resize(lr, [128, 128]).unsqueeze(0).to(device)

    # Model inference
    with torch.no_grad():
        sr = model(lr_up)

    # GÖRSELLEŞTİRME İÇİN BÜYÜT
    lr_vis = TF.resize(lr, [256, 256])
    sr_vis = TF.resize(sr.squeeze(0), [256, 256])

    return (
        lr_vis.permute(1,2,0).cpu().numpy(),
        sr_vis.permute(1,2,0).cpu().numpy()
    )

# =========================
# GRADIO ARAYÜZ
import gradio as gr

interface = gr.Interface(
    fn=super_resolve,
    inputs=gr.Image(type="pil", label="Input Image"),
    outputs=[
        gr.Image(label="Low Resolution (64x64, Blurred)"),
        gr.Image(label="Super-Resolved Output (128x128)")
    ],
    title="Advanced Autoencoder-based Image Super-Resolution",
    description="Low-resolution images are generated using downsampling and Gaussian blur, consistent with the training procedure."
)

interface.launch(share=True)

# MODEL 2
Pooling yerine stride kullanıldı

In [ ]:
# ============================================================
# AUTOENCODER SUPER RESOLUTION (STRIDE-BASED DOWNSAMPLING)
# Pooling YOK – Stride kullanıldı
# Dataset: CelebA (sadece 500 görüntü)
# ============================================================

# =========================
# 1. IMPORTLAR
# =========================
import os
import random
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
import torchvision.transforms.functional as TF

import gradio as gr

# =========================
# 2. DEVICE AYARI
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# 3. DATASET HAZIRLAMA (SADECE 500 GÖRÜNTÜ)
# ============================================================

image_dir = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
all_images = os.listdir(image_dir)

random.seed(42)
selected_images = random.sample(all_images, 500)

target_dir = "/kaggle/working/celeba_500/images"
os.makedirs(target_dir, exist_ok=True)

for img_name in selected_images:
    shutil.copy(
        os.path.join(image_dir, img_name),
        os.path.join(target_dir, img_name)
    )

print("Toplam seçilen görüntü:", len(selected_images))

# ============================================================
# 4. DATASET SINIFI (LR → HR)
# ============================================================

class CelebASRDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.images = os.listdir(image_dir)

        self.hr_transform = T.Compose([
            T.Resize((128, 128)),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        img = Image.open(img_path).convert("RGB")

        hr = self.hr_transform(img)
        lr = TF.resize(hr, [32, 32])   # Low Resolution

        return lr, hr

dataset = CelebASRDataset(target_dir)

# ============================================================
# 5. TRAIN / VAL / TEST SPLIT
# ============================================================

train_size = int(0.7 * len(dataset))
val_size   = int(0.15 * len(dataset))
test_size  = len(dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False)

# ============================================================
# 6. MODEL – STRIDE-BASED AUTOENCODER
# Pooling YOK ❌
# Downsampling = stride=2
# ============================================================

class AutoencoderSR(nn.Module):
    def __init__(self):
        super().__init__()

        # =====================
        # ENCODER (LR → LATENT)
        # =====================
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, stride=2, padding=1),   # 32 → 16
            nn.ReLU(),

            nn.Conv2d(64, 128, 3, stride=2, padding=1), # 16 → 8
            nn.ReLU()
        )

        # =====================
        # DECODER (LATENT → HR)
        # =====================
        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor=2),               # 8 → 16
            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),               # 16 → 32
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),               # 32 → 64
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),               # 64 → 128
            nn.Conv2d(32, 3, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = AutoencoderSR().to(device)

# ============================================================
# 7. LOSS & OPTIMIZER
# ============================================================

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ============================================================
# 8. TRAINING LOOP
# ============================================================

num_epochs = 30
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # -------- TRAIN --------
    model.train()
    train_loss = 0

    for lr_imgs, hr_imgs in train_loader:
        lr_imgs = lr_imgs.to(device)
        hr_imgs = hr_imgs.to(device)

        outputs = model(lr_imgs)
        loss = criterion(outputs, hr_imgs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # -------- VALIDATION --------
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for lr_imgs, hr_imgs in val_loader:
            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            outputs = model(lr_imgs)
            loss = criterion(outputs, hr_imgs)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

# ============================================================
# 9. MODEL KAYDETME
# ============================================================

model_path = "/kaggle/working/autoencoder_stride_sr.pth"
torch.save(model.state_dict(), model_path)
print("Model kaydedildi:", model_path)

# ============================================================
# 10. LOSS GRAFİĞİ
# ============================================================

plt.figure(figsize=(8,5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Training & Validation Loss")
plt.legend()
plt.grid()
plt.show()

# ============================================================
# 11. GRADIO ARAYÜZÜ
# ============================================================


In [ ]:
# ============================================================
# AUTOENCODER SUPER RESOLUTION - SINGLE IMAGE TEST
# Model: Stride-based Autoencoder (32x32 → 128x128)
# Arayüz YOK – Manuel resim test
# ============================================================

import torch
import torch.nn as nn
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
from PIL import Image

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# =========================
# MODEL TANIMI
# =========================
class AutoencoderSR(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, stride=2, padding=1),   # 32 → 16
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1), # 16 → 8
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Upsample(scale_factor=2),  # 8 → 16
            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),  # 16 → 32
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),  # 32 → 64
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),  # 64 → 128
            nn.Conv2d(32, 3, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

# =========================
# MODEL YÜKLE
# =========================
model_path = "/kaggle/working/autoencoder_stride_sr.pth"  # ← gerekirse değiştir
model = AutoencoderSR().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# =========================
# TEST EDİLECEK RESİM
# =========================
test_img_path = "/kaggle/working/celeba_500/images/003540.jpg"  # ← KENDİ RESMİN

img = Image.open(test_img_path).convert("RGB")

# =========================
# HR / LR OLUŞTUR
# =========================
hr = TF.to_tensor(T.Resize((128, 128))(img))
lr = TF.resize(hr, [32, 32])

lr_input = lr.unsqueeze(0).to(device)

# =========================
# SUPER RESOLUTION
# =========================
with torch.no_grad():
    sr = model(lr_input)

# =========================
# GÖRSELLEŞTİRME
# =========================
plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.imshow(lr.permute(1,2,0))
plt.title("Low Resolution (32x32)")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(sr.squeeze(0).permute(1,2,0).cpu())
plt.title("Super Resolved (Model Output)")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(hr.permute(1,2,0))
plt.title("Original / Ground Truth (128x128)")
plt.axis("off")

plt.tight_layout()
plt.show()


# MODEL 3


In [ ]:
# =========================
# IMPORTS & DEVICE
# =========================
import os
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# DATASET (REALISTIC LR)
# =========================
class SuperResolutionDataset(Dataset):
    def __init__(self, image_dir, hr_size=128):
        self.image_dir = image_dir
        self.images = os.listdir(image_dir)

        # High-resolution
        self.hr_transform = transforms.Compose([
            transforms.Resize((hr_size, hr_size)),
            transforms.ToTensor()
        ])

        # Low-resolution (downsample + blur)
        self.lr_transform = transforms.Compose([
            transforms.Resize((hr_size // 2, hr_size // 2)),
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        img = Image.open(img_path).convert("RGB")

        hr = self.hr_transform(img)   # 128x128
        lr = self.lr_transform(img)   # 64x64

        return lr, hr

# =========================
# MODEL
# =========================
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.BatchNorm2d(channels)
        )

    def forward(self, x):
        return x + self.block(x)

class AutoencoderSR_Advanced(nn.Module):
    def __init__(self, num_blocks=5):
        super().__init__()

        self.head = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(inplace=True)
        )

        self.body = nn.Sequential(
            *[ResidualBlock(64) for _ in range(num_blocks)]
        )

        self.upsample = nn.Sequential(
            nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True)
        )

        self.tail = nn.Sequential(
            nn.Conv2d(64, 3, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        feat = self.head(x)
        body_out = self.body(feat)
        up = self.upsample(body_out)
        out = self.tail(up)

        # 🔹 Global Skip Connection
        skip = F.interpolate(x, scale_factor=2, mode="bilinear", align_corners=False)
        return out + skip

# =========================
# DATA PREPARATION
# =========================
image_dir = "/kaggle/working/celeba_500/images"
dataset = SuperResolutionDataset(image_dir)

dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size   = int(0.15 * dataset_size)
test_size  = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=8, shuffle=False)

print(f"Train: {train_size}, Val: {val_size}, Test: {test_size}")

# =========================
# TRAINING
# =========================
model = AutoencoderSR_Advanced().to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20
train_losses = []
val_losses = []

print("Training started")

for epoch in range(num_epochs):
    # ---- TRAIN ----
    model.train()
    train_loss = 0.0

    for lr_imgs, hr_imgs in train_loader:
        lr_imgs = lr_imgs.to(device)
        hr_imgs = hr_imgs.to(device)

        outputs = model(lr_imgs)
        loss = criterion(outputs, hr_imgs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # ---- VALIDATION ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for lr_imgs, hr_imgs in val_loader:
            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            outputs = model(lr_imgs)
            loss = criterion(outputs, hr_imgs)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

# =========================
# SAVE MODEL
# =========================
torch.save(model.state_dict(), "autoencoder_sr_advanced.pth")
print("Model saved.")


In [ ]:
# ============================================================
# AUTOENCODER SUPER RESOLUTION - SINGLE IMAGE TEST
# Model: Stride-based Autoencoder (32x32 → 128x128)
# Arayüz YOK – Manuel resim test
# ============================================================

import torch
import torch.nn as nn
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
from PIL import Image

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Doğru model sınıfı

# =========================
# MODEL TANIMI
# =========================
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.BatchNorm2d(channels)
        )

    def forward(self, x):
        return x + self.block(x)

# =========================
# MODEL YÜKLE
# =========================
# Doğru model sınıfı
model = AutoencoderSR_Advanced().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()


# =========================
# TEST EDİLECEK RESİM
# =========================
test_img_path = "/kaggle/working/celeba_500/images/003540.jpg"  # ← Kendi seçtiğin resim
img = Image.open(test_img_path).convert("RGB")

# =========================
# HR / LR OLUŞTUR
# =========================
hr = TF.to_tensor(T.Resize((128, 128))(img))   # Ground truth
lr = TF.resize(hr, [32, 32])                   # Low-resolution

lr_input = lr.unsqueeze(0).to(device)          # Batch dimension ekle

# =========================
# SUPER RESOLUTION
# =========================
with torch.no_grad():
    sr = model(lr_input)

# =========================
# GÖRSELLEŞTİRME
# =========================
plt.figure(figsize=(12,4))

# Low Resolution
plt.subplot(1,3,1)
plt.imshow(lr.permute(1,2,0).cpu())
plt.title("Low Resolution (32x32)")
plt.axis("off")

# Super Resolved
plt.subplot(1,3,2)
plt.imshow(sr.squeeze(0).permute(1,2,0).cpu())
plt.title("Super Resolved (Model Output)")
plt.axis("off")

# High Resolution / Ground Truth
plt.subplot(1,3,3)
plt.imshow(hr.permute(1,2,0).cpu())
plt.title("Original / Ground Truth (128x128)")
plt.axis("off")

plt.tight_layout()
plt.show()


# MODEL 4

In [ ]:
# ============================================================
# AUTOENCODER SUPER RESOLUTION - FULL BLOCK (TRAIN + SINGLE IMAGE TEST)
# ============================================================

import os
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import matplotlib.pyplot as plt

# =========================
# DEVICE
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# DATASET
# =========================
class SuperResolutionDataset(Dataset):
    def __init__(self, image_dir, hr_size=128):
        self.image_dir = image_dir
        self.images = os.listdir(image_dir)

        # HR Transform
        self.hr_transform = transforms.Compose([
            transforms.Resize((hr_size, hr_size)),
            transforms.ToTensor()
        ])

        # 🔥 Daha gerçekçi LR Transform
        self.lr_transform = transforms.Compose([
            transforms.Resize((hr_size // 2, hr_size // 2)),
            transforms.GaussianBlur(kernel_size=3),
            transforms.ColorJitter(brightness=0.1, contrast=0.1),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.image_dir, self.images[idx])).convert("RGB")
        hr = self.hr_transform(img)
        lr = self.lr_transform(img)
        return lr, hr

# =========================
# MODEL
# =========================
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.BatchNorm2d(channels)
        )

    def forward(self, x):
        return x + self.block(x)

class AutoencoderSR_Advanced(nn.Module):
    def __init__(self, num_blocks=8):
        super().__init__()
        self.head = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.body = nn.Sequential(
            *[ResidualBlock(64) for _ in range(num_blocks)]
        )
        self.upsample = nn.Sequential(
            nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True)
        )
        self.tail = nn.Sequential(
            nn.Conv2d(64, 3, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.head(x)
        x = self.body(x)
        x = self.upsample(x)
        x = self.tail(x)
        return x

# =========================
# EDGE LOSS
# =========================
def edge_loss(sr, hr):
    sobel_x = torch.tensor([[1,0,-1],[2,0,-2],[1,0,-1]],
                            dtype=torch.float32).view(1,1,3,3).to(sr.device)
    sobel_y = torch.tensor([[1,2,1],[0,0,0],[-1,-2,-1]],
                            dtype=torch.float32).view(1,1,3,3).to(sr.device)

    def sobel(img):
        gray = img.mean(dim=1, keepdim=True)
        gx = F.conv2d(gray, sobel_x, padding=1)
        gy = F.conv2d(gray, sobel_y, padding=1)
        return torch.sqrt(gx**2 + gy**2)

    return F.l1_loss(sobel(sr), sobel(hr))

# =========================
# DATA PREPARATION
# =========================
image_dir = "/kaggle/working/celeba_500/images"
dataset = SuperResolutionDataset(image_dir)

train_size = int(0.7 * len(dataset))
val_size   = int(0.15 * len(dataset))
test_size  = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)

# =========================
# MODEL / OPTIMIZER / LOSS
# =========================
model = AutoencoderSR_Advanced().to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 50
train_losses, val_losses = [], []

print("🚀 Training Started")

for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    for lr_imgs, hr_imgs in train_loader:
        lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
        sr = model(lr_imgs)
        loss = criterion(sr, hr_imgs) + 0.1 * edge_loss(sr, hr_imgs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for lr_imgs, hr_imgs in val_loader:
            lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
            sr = model(lr_imgs)
            loss = criterion(sr, hr_imgs)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}] | Train: {train_loss:.4f} | Val: {val_loss:.4f}")

# =========================
# LOSS GRAFİĞİ
# =========================
plt.figure(figsize=(8,5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

# =========================
# MODEL KAYDET
# =========================
torch.save(model.state_dict(), "sr_autoencoder_final.pth")
print("✅ Model saved.")

# =========================
# TEK RESİM TESTİ
# =========================
test_img_path = "/kaggle/working/celeba_500/images/003540.jpg"
img = Image.open(test_img_path).convert("RGB")

hr = TF.to_tensor(transforms.Resize((128,128))(img))
lr = TF.resize(hr, [64,64])

lr_input = lr.unsqueeze(0).to(device)

with torch.no_grad():
    sr = model(lr_input)

# =========================
# GÖRSELLEŞTİRME
# =========================
plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.imshow(lr.permute(1,2,0).cpu())
plt.title("Low Resolution (64x64)")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(sr.squeeze(0).permute(1,2,0).cpu())
plt.title("Super Resolved")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(hr.permute(1,2,0).cpu())
plt.title("Original / HR (128x128)")
plt.axis("off")

plt.tight_layout()
plt.show()
